In [ ]:
import numpy as np

## 1. Создаем искусственный нейрон со случайными значениями
Реализуем математическую модель искусственного нейрона на основе формулы:
<img src="images/neuron_formula.png">

In [12]:
def neuron_rand(input_size):
    # Входной вектор:
    x = np.random.uniform(low=-1, high=1, size=input_size)

    # Вектор весов:
    w = np.random.uniform(low=-1, high=1, size=input_size)

    # Смещение (байес):
    b = np.random.uniform(low=-1, high=1, size=1)[0]

    # Вычисляем выход:
    y = np.sum(x*w) + b

    # Добавляем нелинейность (функция активации):
    y = np.sign(y)
    
    return y


for i in range(5):
    print (neuron_rand(10))

1.0
-1.0
-1.0
1.0
-1.0


## 2. Класс искусственного нейрона
Реализуем класс искусственного нейрона для реализации логических операторов

In [13]:
class Neuron:
    def __init__(self, input_size, w, b=1, activation=np.sign):
        self.w = w if isinstance(w, np.ndarray) else np.array(w)    # вектор весов
        self.b = b                                                  # байес
        self.activation = activation                                # функция активации
    
    def __call__(self, x):
        y = np.sum(x*self.w) + self.b
        y = self.activation(y)
        return y

In [14]:
# Логическое НЕ (отрицание):
neuron = Neuron(1, w=[-1], b=0.5)

x = [1]
y = neuron(x)
print (f'Logic NOT: x = {x}, y = {y}')

x = [-1]
y = neuron(x)
print (f'Logic NOT: x = {x}, y = {y}')

Logic NOT: x = [1], y = -1.0
Logic NOT: x = [-1], y = 1.0


In [15]:
# Логическое И (конъюнкция):
neuron = Neuron(2, w=[1,1], b=-1.5)

x = [1,1]
y = neuron(x)
print (f'Logic AND: x = {x}, y = {y}')

x = [0,0]
y = neuron(x)
print (f'Logic AND: x = {x}, y = {y}')

x = [0,1]
y = neuron(x)
print (f'Logic AND: x = {x}, y = {y}')

x = [1,0]
y = neuron(x)
print (f'Logic AND: x = {x}, y = {y}')

Logic AND: x = [1, 1], y = 1.0
Logic AND: x = [0, 0], y = -1.0
Logic AND: x = [0, 1], y = -1.0
Logic AND: x = [1, 0], y = -1.0


In [16]:
# Логическое ИЛИ (дизъюнкция):
neuron = Neuron(2, w=[1,1], b=-0.5)

x = [1,1]
y = neuron(x)
print (f'Logic OR: x = {x}, y = {y}')

x = [0,0]
y = neuron(x)
print (f'Logic OR: x = {x}, y = {y}')

x = [0,1]
y = neuron(x)
print (f'Logic OR: x = {x}, y = {y}')

x = [1,0]
y = neuron(x)
print (f'Logic OR: x = {x}, y = {y}')

Logic OR: x = [1, 1], y = 1.0
Logic OR: x = [0, 0], y = -1.0
Logic OR: x = [0, 1], y = 1.0
Logic OR: x = [1, 0], y = 1.0


## 3. Класс нейронной сети
Реализуем класс простой нейронной сети прямого распространения
<img src="images/XOR.png">

In [ ]:
def ReLU(x): return max(0, x)
def create_neuron_NOT(): return Neuron(2, w=[-1], b=0.5, activation=ReLU)
def create_neuron_AND(): return Neuron(2, w=[1,1], b=-1.5, activation=ReLU)
def create_neuron_OR(): return Neuron(2, w=[1,1], b=-0.5)

class FeedForwardNetwork:
    def __init__(self, neurons, connections, output_node, verbose=False):
        self.neurons = neurons
        self.connections = connections
        self.output_node = output_node
        self.verbose = verbose
        
    def fire(self, node, outputs):
        neuron = self.neurons[node]
        inputs = self.connections[node]
        
        if self.verbose:
            print (f'node={node}, inputs={inputs}, outputs={outputs}')
        
        for x in inputs:
            if x not in outputs:
                outputs[x] = self.fire(x, outputs)
        
        return neuron([outputs[x] for x in inputs])
        
    def __call__(self, x):
        outputs = {}
        for node, value in x.items():
            outputs[node] = value
            
        outputs[self.output_node]  = self.fire(self.output_node, outputs)
        
        if self.verbose:
            print (f'Outputs: {outputs}')
        
        return outputs[self.output_node]

In [17]:
# XOR из 5 нейронов:
neurons = {
    'NOT_1_1': create_neuron_NOT(),
    'NOT_1_2': create_neuron_NOT(),
    'AND_2_1': create_neuron_AND(),
    'AND_2_2': create_neuron_AND(),
    'OR_3_1': create_neuron_OR()
}

connections = {
    'NOT_1_1': ('x1',),
    'NOT_1_2': ('x2',),
    'AND_2_1': ('NOT_1_1', 'x2'),
    'AND_2_2': ('NOT_1_2', 'x1'),
    'OR_3_1': ('AND_2_1', 'AND_2_2')
}

# Инициализируем нейронную сеть:
ffn_XOR = FeedForwardNetwork(neurons, connections, output_node='OR_3_1', verbose=False)

# Тестируем:
x = {'x1': 1, 'x2': 1}
y = ffn_XOR(x)
print (f'x={x}, y={y}')

x = {'x1': -1, 'x2': -1}
y = ffn_XOR(x)
print (f'x={x}, y={y}')

x = {'x1': 1, 'x2': -1}
y = ffn_XOR(x)
print (f'x={x}, y={y}')

x = {'x1': -1, 'x2': 1}
y = ffn_XOR(x)
print (f'x={x}, y={y}')

x={'x1': 1, 'x2': 1}, y=-1.0
x={'x1': -1, 'x2': -1}, y=-1.0
x={'x1': 1, 'x2': -1}, y=1.0
x={'x1': -1, 'x2': 1}, y=1.0
